In [1]:
from autogen_agentchat.agents import CodingAssistantAgent, ToolUseAssistantAgent, AssistantAgent
from autogen_agentchat.task import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_core.components.tools import FunctionTool
from autogen_ext.models import AzureOpenAIChatCompletionClient

import os
from dotenv import load_dotenv

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Access environment variables
AZURE_OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
AZURE_OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')
AZURE_OPENAI_DEPLOYMENT = os.getenv('AZURE_OPENAI_DEPLOYMENT')

GOOGLE_SEARCH_ENGINE_ID=os.getenv('GOOGLE_SEARCH_ENGINE_ID')
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')

In [3]:
client = AzureOpenAIChatCompletionClient(
    model=AZURE_OPENAI_DEPLOYMENT,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_key=AZURE_OPENAI_API_KEY,
    model_capabilities={
        "vision":True,
        "function_calling":True,
        "json_output":True,
    }
)

In [4]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    api_key = os.getenv("GOOGLE_API_KEY")
    search_engine_id = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

    if not api_key or not search_engine_id:
        raise ValueError("API key or Search Engine ID not found in environment variables")

    url = "https://www.googleapis.com/customsearch/v1"
    params = {"key": str(api_key), "cx": str(search_engine_id), "q": str(query), "num": str(num_results)}

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("items", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item["snippet"], "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    return enriched_results

In [5]:
def analyze_stock(ticker: str) -> dict:  # type: ignore[type-arg]
    import os
    from datetime import datetime, timedelta

    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import yfinance as yf
    from pytz import timezone  # type: ignore

    stock = yf.Ticker(ticker)

    # Get historical data (1 year of data to ensure we have enough for 200-day MA)
    end_date = datetime.now(timezone("UTC"))
    start_date = end_date - timedelta(days=365)
    hist = stock.history(start=start_date, end=end_date)

    # Ensure we have data
    if hist.empty:
        return {"error": "No historical data available for the specified ticker."}

    # Compute basic statistics and additional metrics
    current_price = stock.info.get("currentPrice", hist["Close"].iloc[-1])
    year_high = stock.info.get("fiftyTwoWeekHigh", hist["High"].max())
    year_low = stock.info.get("fiftyTwoWeekLow", hist["Low"].min())

    # Calculate 50-day and 200-day moving averages
    ma_50 = hist["Close"].rolling(window=50).mean().iloc[-1]
    ma_200 = hist["Close"].rolling(window=200).mean().iloc[-1]

    # Calculate YTD price change and percent change
    ytd_start = datetime(end_date.year, 1, 1, tzinfo=timezone("UTC"))
    ytd_data = hist.loc[ytd_start:]  # type: ignore[misc]
    if not ytd_data.empty:
        price_change = ytd_data["Close"].iloc[-1] - ytd_data["Close"].iloc[0]
        percent_change = (price_change / ytd_data["Close"].iloc[0]) * 100
    else:
        price_change = percent_change = np.nan

    # Determine trend
    if pd.notna(ma_50) and pd.notna(ma_200):
        if ma_50 > ma_200:
            trend = "Upward"
        elif ma_50 < ma_200:
            trend = "Downward"
        else:
            trend = "Neutral"
    else:
        trend = "Insufficient data for trend analysis"

    # Calculate volatility (standard deviation of daily returns)
    daily_returns = hist["Close"].pct_change().dropna()
    volatility = daily_returns.std() * np.sqrt(252)  # Annualized volatility

    # Create result dictionary
    result = {
        "ticker": ticker,
        "current_price": current_price,
        "52_week_high": year_high,
        "52_week_low": year_low,
        "50_day_ma": ma_50,
        "200_day_ma": ma_200,
        "ytd_price_change": price_change,
        "ytd_percent_change": percent_change,
        "trend": trend,
        "volatility": volatility,
    }

    # Convert numpy types to Python native types for better JSON serialization
    for key, value in result.items():
        if isinstance(value, np.generic):
            result[key] = value.item()

    # Generate plot
    plt.figure(figsize=(12, 6))
    plt.plot(hist.index, hist["Close"], label="Close Price")
    plt.plot(hist.index, hist["Close"].rolling(window=50).mean(), label="50-day MA")
    plt.plot(hist.index, hist["Close"].rolling(window=200).mean(), label="200-day MA")
    plt.title(f"{ticker} Stock Price (Past Year)")
    plt.xlabel("Date")
    plt.ylabel("Price ($)")
    plt.legend()
    plt.grid(True)

    # Save plot to file
    os.makedirs("coding", exist_ok=True)
    plot_file_path = f"coding/{ticker}_stockprice.png"
    plt.savefig(plot_file_path)
    print(f"Plot saved as {plot_file_path}")
    result["plot_file_path"] = plot_file_path

    return result

In [6]:
google_search_tool = FunctionTool(
    google_search,
    description="Search Google for information, returns results with a snippet and body content"
    )
stock_analysis_tool = FunctionTool(
    analyze_stock,
    description="Analyze stock data and generate a plot"
    )

In [7]:
search_agent = AssistantAgent(
    name="Google_Search_Agent",
    model_client=client,
    tools=[google_search_tool],
    description="Search Google for information, returns top 2 results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

stock_analysis_agent = AssistantAgent(
    name="Stock_Analysis_Agent",
    model_client=client,
    tools=[stock_analysis_tool],
    description="Analyze stock data and generate a plot",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=client,
    description="Generate a report based on the search and stock analysis results",
    system_message="You are a helpful assistant that can generate a comprehensive report on a given topic based on search and stock analysis. When you done with generating the report, reply with TERMINATE.",
)

termination = TextMentionTermination("TERMINATE")


In [8]:
team = RoundRobinGroupChat(
    [search_agent, stock_analysis_agent, report_agent],
    termination_condition=termination
    )

In [15]:
result = await team.run(task="Write a financial report on American airlines")

In [19]:
from IPython.display import Markdown, display, JSON

total_prompt_tokens = 0
total_completion_tokens = 0

for index, message in enumerate(result.messages, start=1):
    display(Markdown('<hr>'))
    text = f"## Step: {index} (output from {message.source})"
    display(Markdown(text))

    if message.models_usage:
        print(f'Prompt Tokens: {message.models_usage.prompt_tokens}')
        print(f'Completion Tokens: {message.models_usage.completion_tokens}')
        total_prompt_tokens += message.models_usage.prompt_tokens
        total_completion_tokens += message.models_usage.completion_tokens
    display(Markdown('<hr>'))

    if isinstance(message.content, str):
        display(Markdown(message.content))
    else:
        for item in message.content:
            display(JSON(item.__dict__, expanded=True))

display(Markdown('<hr>'))
print(f'Total Prompt Tokens: {total_prompt_tokens}')
print(f'Total Completion Tokens: {total_completion_tokens}')
print(f'Total Tokens: {total_prompt_tokens + total_completion_tokens}')

<hr>

## Step: 1 (output from user)

<hr>

Write a financial report on American airlines

<hr>

...

Prompt Tokens: 4314
Completion Tokens: 700


<hr>

### American Airlines Financial Report Q3 2023

#### Executive Summary
American Airlines Group Inc. (NASDAQ: AAL) reported strong financial results for the third quarter of 2023, marked by a substantial increase in revenue and effective cost management strategies. Amidst the ongoing recovery of the global aviation industry, the airline has strategically positioned itself to leverage increasing travel demand while navigating economic and operational challenges.

#### Financial Performance

1. **Revenue and Profitability:**
   - **Revenue Growth:** The airline reported revenues of approximately $13.5 billion for Q3 2023, reflecting a significant upturn fueled by robust passenger demand and strategic pricing initiatives.
   - **Net Income:** The company posted a GAAP profit, underscoring successful cost containment measures and efficient operational execution despite pressures such as rising fuel costs.

2. **Operating Expenses:**
   - **Fuel and Labor Costs:** The company continues to manage substantial expenditures in fuel and labor. Through hedging and efficiency initiatives, American Airlines has been able to mitigate the impacts of fuel price volatility.
   - **Operational Efficiency:** Cost-saving measures include fleet optimization and operational streamlining, contributing to better management of variable costs.

3. **Cash Flow and Liquidity:**
   - American Airlines has sustained a strong liquidity position, ensuring adequate cash flow to support operations and ongoing investments. This financial stability is vital for maintaining competitive advantage and operational resilience.

#### Strategic Initiatives

1. **Fleet Modernization and Network Expansion:**
   - The airline remains committed to their fleet renewal program, integrating more fuel-efficient aircraft to improve economic performance and reduce environmental impact. Network expansion, particularly in recovering international markets, is a key focus to capture growing demand.

2. **Enhanced Customer Experience:**
   - Investments in digital technologies and upgrades to the customer experience, such as improved booking systems and in-flight services, have been prioritized. Additionally, modifications to loyalty programs aim to strengthen passenger retention and satisfaction.

3. **Financial Discipline and Debt Management:**
   - Debt reduction and enhanced capital structure have been targets for American Airlines, with strategic refinancing and cost control underpinning financial stability and operational agility.

#### Challenges and Opportunities

1. **Challenges:**
   - **Economic and Geopolitical Factors:** Challenges such as fluctuating currencies and geopolitical tensions can impact travel patterns and operational costs. The company remains vigilant in mitigating these risks.
   - **Regulatory Compliance:** The airline is adapting to stricter sustainability regulations, focusing on meeting environmental standards through investments in green technologies and practices.

2. **Opportunities:**
   - **Market Recovery and Expansion:** The rebound of international and particularly long-haul travel presents opportunities for revenue growth and network diversification.
   - **Sustainability Leadership:** The push for industry sustainability provides an avenue for innovation in sustainable aviation fuels and emissions reduction technologies.

#### Conclusion

American Airlines has demonstrated strong financial and operational performance during Q3 2023, with a strategic emphasis on cost management, customer experience enhancement, and sustainability. Moving forward, the company is well-positioned to capitalize on the recovering travel market and emerging opportunities in environmental sustainability.

For comprehensive financial details and updated strategic plans, stakeholders are advised to consult American Airlines' official releases and detailed financial statements available via their [Investor Relations page](https://americanairlines.gcs-web.com/).

This report synthesizes data and insights as of Q3 2023, reflecting American Airlines' performance and strategic approach amid evolving industry dynamics.

TERMINATE

<hr>

Total Prompt Tokens: 15360
Total Completion Tokens: 1987
Total Tokens: 17347
